# <center>Big Data For Engineers &ndash; Exercises</center>
## <center>Spring 2020 &ndash; Week 0 &ndash; ETH Zurich</center>

## 1. Jupyter Basics

Welcome to this Jupyter notebook. Jupyter is a web-based open-source tool based on Python that allows you to run python (and other types of) code, visualize results and discuss results, and organize everything into notebooks like this one. We use the notebook server on Microsoft Azure, but you can also install your own.
A notebook is organized in cells. Cells of this notebook contain Python code (but other cell types exists). To run a cell, select it, then press ctrl+enter. Try it out!

In [ ]:
print("Hello World")

By default, the last expression is printed. Like this:

In [ ]:
maxX = 10
[x * x for x in range(maxX)]

You can also edit the text. Just double-click on a cell. It's made with markdown code. After you are done editing, press ctrl+enter

We will do about half of the exercises with Jupyter. You will learn most things as we go. The notebook you are seing is your copy stored on your account (including the output of the cells). Write your answers inline and save regularly. (Also save a copy somewhere else from time to time &ndash; this service is still beta.)

### Read more:
* [Jupyter](http://jupyter.org/)
* [Learn Python](http://www.learnpython.org)
* [Python documentation](https://docs.python.org/3/)

## 2. Bash Scripts

Code blocks by default are executed using a python interpreter (for a python notebook, such as this one).

Other languages can be used with annotations.
For instance, a code block can be converted into a bash code block using ```%%bash``` at the beginning:


In [ ]:
%%bash
echo "Test File" > test_file
cat test_file

Note: do not expect files you write into this file system to be durable. The sandboxed environment may be reset and data lost when notebook is closed. The notebook files themselves are durable though.

Bash commands can be also inlined using exclamation mark (```!```) infront of the bash line

In [ ]:
print("First I modify the file.")

!echo "Hello World" > test_file

print("Then I check its content:")
print()

!cat test_file

## 3. Extensions

As part of this course you will use Jupyter to interact with various systems and interfaces (e.g. SQL, Map Reduce, Spark). To use these, you will need to install certain Python and Jupyter extensions. Note, that as like the file system the extensions will not be durable and need to be rerun when the notebook server is restarted.

#### 3.1. Setting up a SQL connection

As part of preparation for the next week's exercise, let's setup a connection to a sample relational database (hosted from this course's Azure account)

First set the access variables (make sure you execute the next code block by running ctrl+enter)


In [ ]:
server='ethbigdata2019.database.windows.net'
user='student@ethbigdata2019'
password='BigData19'
database='beer.stackexchange.com'

Next, install the [PyMSSQL](http://www.pymssql.org) python package. Run the following cell and make sure that the output confirms that the installation was successfull.

In [ ]:
!pip install pymssql==2.1.2

#### 3.2. Running a SQL Query
Then we run a first query against our server (following [this tutorial](https://msdn.microsoft.com/library/mt715796.aspx?f=255&MSPPError=-2147217396) from the Azure website). Make sure that running the following cell successfully prints some rows.

In [ ]:
import pymssql
import os

os.environ['TDSVER'] = '7.3'

conn = pymssql.connect(server=server, user=user,
                       password=password, database=database)
cursor = conn.cursor(as_dict=True)

cursor.execute('SELECT TOP 10 Id, DisplayName FROM Users ORDER BY Id')

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

#### 3.3. Inlining SQL 
It is also possible to inline SQL code. To do this install the following Jupyter extension:

In [ ]:
!git clone https://github.com/catherinedevlin/ipython-sql.git; \
cd ipython-sql && LC_CTYPE="C.UTF-8" python setup.py -q install

> <span style="color:red;">**At this point, we need to restart the notebook kernel!**</span>

To do that, go to *Kernel* in the menu at the top, then click on *Restart*. You need to run the cell with the connection details again after that.

We can now load the extension and establish a connection to our database from above. Run the following cell and make sure the output says *"Connected: &lt;connection string&gt;"*.

In [ ]:
%load_ext sql
connection_string = 'mssql+pymssql://{user}:{password}@{server}:1433/{database}'.format(
        server=server, user=user, password=password, database=database)
print(connection_string)
%sql $connection_string

Now we can use the ```%sql``` and ```%%sql``` magic words to run SQL directly. ```%%sql``` makes a cell a SQL cell. A SQL cell can run an arbitrary number of SQL statements and displays the result of the last one of them.

In [ ]:
%%sql 
SELECT TOP 10 Id, DisplayName FROM Users ORDER BY Id

The ```%sql``` magic words lets us run SQL statements in a regular cell. Again, the result of the last statement is displayed.

In [ ]:
print("Run a query!")
%sql SELECT TOP 10 Id, DisplayName FROM Users ORDER BY Id;

#### 3.4. Plotting results
Matplotlib can also be used to plot results. Next is a plot of a sample query that finds the number of created users per year.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

result = %sql SELECT YEAR(CreationDate) as CreationYear, COUNT(*) as YearCount \
                FROM Users \
                GROUP BY YEAR(CreationDate) \
                ORDER BY YEAR(CreationDate) ASC;
            
# Print the result in tabular form
print(result)
            
# Convert the result to a Pandas data frame
df = result.DataFrame()

# Extract x and y values for a plot
x = df['CreationYear'].tolist()
y = df['YearCount'].tolist()

# Plot the distribution of registrations per year
fig, ax = plt.subplots()
ax.bar(range(len(df.index)), y, tick_label=[int(i) for i in x], align='center')
ax.set_xlabel('Creation Year')
ax.set_ylabel('Number of Users')